In [1]:
# ! pip install sqlalchemy
# ! pip install pymysql

In [2]:
import pandas as pd
from sqlalchemy import create_engine 
import pymysql.cursors
import os
# import getpass
import urllib.parse
from datetime import datetime

In [3]:
urllib.parse.quote_plus('!') # parse to 'remove-change' the weird characters in our password, here it is '!'

'%21'

In [4]:
urllib.parse.quote_plus('é')

'%C3%A9'

In [5]:
pw_raw = 'Alsimar10' # os.getenv('mysql_andy') # I had to put my.sql password
pw = urllib.parse.quote_plus(pw_raw)

In [8]:
#engine = sqlalchemy.create_engine(f"mysql://{username}:{password}@{host}:{port}")

connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/sakila' # we connect to our computer, if we connect to a company
                     # this is the driver                             # data base, then we need an IP address
engine = create_engine(connection_string)

In [9]:
with engine.connect() as conn:                                   # we are creating an alias
    conn.execute(f"CREATE DATABASE IF NOT EXISTS sakila") # connecting to sakila

In [19]:
sql = """select *, year(rental_date) as year_query, month(rental_date) as month_query
    from sakila.rental
"""

date_df = pd.read_sql_query(sql, engine)
date_df


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,year_query,month_query
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,2005,5
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,2005,5
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,2005,5
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,2005,5
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,2005,5
...,...,...,...,...,...,...,...,...,...
16039,16045,2005-08-23 22:25:26,772,14,2005-08-25 23:54:26,1,2006-02-15 21:30:53,2005,8
16040,16046,2005-08-23 22:26:47,4364,74,2005-08-27 18:02:47,2,2006-02-15 21:30:53,2005,8
16041,16047,2005-08-23 22:42:48,2088,114,2005-08-25 02:48:48,2,2006-02-15 21:30:53,2005,8
16042,16048,2005-08-23 22:43:07,2019,103,2005-08-31 21:33:07,1,2006-02-15 21:30:53,2005,8


In [23]:
date_df[['year_query', 'month_query']]

,year_query,month_query
0,2005,5
1,2005,5
2,2005,5
3,2005,5
4,2005,5
...,...,...
16039,2005,8
16040,2005,8
16041,2005,8
16042,2005,8


In [16]:
# creating a function

def rentals_month(engine, month, year):
    """Executes a SQL query to retrieve the rental data for the specified month and year from the
    rental table in the Sakila database, and return it as a pandas DataFrame """
    
    selected_y = year
    selected_m = month
    
    sql = """select *, year(rental_date) as year_query, month(rental_date) as month_query
    from sakila.rental
    """

    date_df = pd.read_sql_query(sql, engine)
    
    mask = (date_df['year_query']==selected_y) & (date_df['month_query']==selected_m)
    
    return date_df[mask]
   


In [51]:
df = rentals_month(engine, 8, 2005)
df['nouveau']= pd.Series(df['customer_id'].value_counts().groupby(['customer_id']))
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,year_query,month_query,nouveau
10176,10181,2005-08-01 00:00:44,3111,113,2005-08-04 19:33:44,1,2006-02-15 21:30:53,2005,8,NaN
10177,10182,2005-08-01 00:08:01,4112,578,2005-08-09 18:14:01,2,2006-02-15 21:30:53,2005,8,NaN
10178,10183,2005-08-01 00:08:01,4319,377,2005-08-09 20:41:01,1,2006-02-15 21:30:53,2005,8,NaN
10179,10184,2005-08-01 00:09:33,2785,77,2005-08-05 04:12:33,2,2006-02-15 21:30:53,2005,8,NaN
10180,10185,2005-08-01 00:12:11,1266,64,2005-08-03 03:03:11,1,2006-02-15 21:30:53,2005,8,NaN
...,...,...,...,...,...,...,...,...,...,...
16039,16045,2005-08-23 22:25:26,772,14,2005-08-25 23:54:26,1,2006-02-15 21:30:53,2005,8,NaN
16040,16046,2005-08-23 22:26:47,4364,74,2005-08-27 18:02:47,2,2006-02-15 21:30:53,2005,8,NaN
16041,16047,2005-08-23 22:42:48,2088,114,2005-08-25 02:48:48,2,2006-02-15 21:30:53,2005,8,NaN
16042,16048,2005-08-23 22:43:07,2019,103,2005-08-31 21:33:07,1,2006-02-15 21:30:53,2005,8,NaN


In [54]:
df['customer_id'].value_counts()

customer_id
148    18
569    18
119    18
15     18
266    17
       ..
498     3
191     2
164     2
281     2
318     1
Name: count, Length: 599, dtype: int64

In [55]:
def rental_count_month (rental_func, engine, month, year):
    """Input along with the month and year and returns a new DataFrame containing the number of rentals made by 
    each customer_id during the selected month and year"""
    
    df = rental_func(engine, month, year)
    sr_temp = df['customer_id'].value_counts() # convertir  en dataframe
    df2 = pd.DataFrame(sr_temp).reset_index()
    
    return df2


In [61]:
df1 = rental_count_month (rentals_month, engine, 7, 2005)
df2 = rental_count_month (rentals_month, engine, 8, 2005)

In [62]:
def compare_rentals(df1, df2):
    """return a combined DataFrame with a new 'difference' column, which is the difference 
    between the number of rentals in the two months"""
    df1= df1.rename(columns={'count': 'month1_count'})
    df2= df2.rename(columns = {'count': 'month2_count'})
    df3 = pd.merge(df1, df2, how = 'outer', on = df1['customer_id'])
    df3['difference']= df3['month1_count']-df3['month2_count']
    return df3


In [63]:
compare_rentals(df1, df2)

,key_0,customer_id_x,month1_count,customer_id_y,month2_count,difference
0,148,148,22,148,18,4
1,102,102,21,569,18,3
2,75,75,20,119,18,2
3,236,236,20,15,18,2
4,91,91,19,266,17,2
...,...,...,...,...,...,...
594,554,554,4,498,3,1
595,514,514,4,191,2,2
596,150,150,4,164,2,2
597,234,234,3,281,2,1
